In [1]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
	using Flux: @functor
	using ..Transformers: Abstract3DTensor, Container, epsilon, batchedmul, batched_triu!
	using NNlib
	using Tullio, ForwardDiff, Zygote
	using Einsum
end

In [2]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end


Vocabulary{String}(13, unk=0)

In [3]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [4]:
#nichts ändern, generieren 10 string token, nutze 1-10

sample_data()

(["10", "4", "3", "9", "9", "6", "3", "3", "4", "7"], ["10", "4", "3", "9", "9", "6", "3", "3", "4", "7"])

In [5]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [6]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data, +3
    end

sample_ex = preprocess.(sample_data()) = (["11", "2", "9", "5", "2", "6", "6", "10", "9", "7", "8", "12"], ["11", "2", "9", "5", "2", "6", "6", "10", "9", "7", "8", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 5, 12, 8, 5, 9, 9, 13, 12, 10, 11, 3]


12-element Vector{Int64}:
  2
  5
 12
  8
  5
  9
  9
 13
 12
 10
 11
  3

In [7]:
#nichts ändern
sample = preprocess.(sample_data())


(["11", "6", "8", "1", "6", "3", "9", "8", "4", "4", "1", "12"], ["11", "6", "8", "1", "6", "3", "9", "8", "4", "4", "1", "12"])

In [8]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
  9
 11
  4
  9
  6
 12
 11
  7
  7
  4
  3

In [9]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

┌ Info: The GPU function is being called but the GPU is not accessible. 
│ Defaulting back to the CPU. (No action is required if you want to run on the CPU).
└ @ Flux /Users/johannes/.julia/packages/Flux/qAdFM/src/functor.jl:187


Embed(512)

In [10]:
#define a position embedding layer mentioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu


PositionEmbedding(512)

In [11]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [12]:
abstract type AbstractAttention end

In [13]:
create_atten_mask1(T::Type, score::AbstractArray, ::Nothing, future) = create_atten_mask1(T, score, fill!(similar(score, size(score,1), size(score, 2), 1), one(T)), future)
function create_atten_mask1(T::Type, score::AbstractArray, _mask::AbstractArray, future::Bool=false)
  #size(mask) == (q, k, n, b)

  # ql, kl = size(mask)
  mask = copy(_mask)

  maskval = convert(T, -1e9)
  !future && batched_triu!(mask, 0)
  mask .= (1 .- mask) .* maskval
  return mask
end

create_atten_mask1 (generic function with 3 methods)

In [14]:
Flux.@nograd create_atten_mask1

In [15]:
struct MultiheadAttention_1{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end

In [16]:
Flux.functor(mh::MultiheadAttention_1) = (mh.iqproj, mh.ikproj, mh.ivproj, mh.oproj), m -> MultiheadAttention_1(mh.head, mh.future, m..., mh.drop)

In [17]:
MultiheadAttention_1(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = MultiheadAttention_1(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, mh::MultiheadAttention_1)
    hs = div(size(mh.iqproj.weight)[1], mh.head)
    is = size(mh.iqproj.weight)[end]
    os = size(mh.oproj.weight)[1]

    print(io, "MultiheadAttention(")
    print(io, "head=$(mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(mh.drop.p))")
    else
        print(io, ")")
    end
end



In [18]:
function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}
qs = size(query)
ks = size(key)
vs = size(value)

#size(ipq) == (h, q_seq_len, batch)
ipq = @toNd mh.iqproj(query)
ipk = @toNd mh.ikproj(key)
ipv = @toNd mh.ivproj(value)

h = size(ipq, 1)
hs = div(h, mh.head)

#size(ipq) == (hs, q_seq_len, head, batch)
ipq = permutedims(reshape(ipq, hs, mh.head, qs[2], qs[3]), [1, 3, 2, 4])
ipk = permutedims(reshape(ipk, hs, mh.head, ks[2], ks[3]), [1, 3, 2, 4])
ipv = permutedims(reshape(ipv, hs, mh.head, vs[2], vs[3]), [1, 3, 2, 4])

#size(ipq) == (hs, q_seq_len, head * batch)
ipq = reshape(ipq, hs, qs[2], :)
ipk = reshape(ipk, hs, ks[2], :)
ipv = reshape(ipv, hs, vs[2], :)

atten = attention1(ipq,ipk,ipv,
mask,
mh.future,
mh.drop)

atten = permutedims(reshape(atten, hs, qs[2], mh.head, qs[3]), [1, 3, 2, 4]) #size(atten) == (hs, head, ql, b)
atten = reshape(atten, h, qs[2], qs[3]) #size(atten) == (h, ql, b)

out = @toNd mh.oproj(atten)
out #size(out) == (h, q_seq_len, batch)
end

function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: AbstractMatrix{T},
                         A2 <: AbstractMatrix{T},
                         A3 <: AbstractMatrix{T}}

# size(query) == (dims, seq_len)
ipq = mh.iqproj(query)
ipk = mh.ikproj(key)
ipv = mh.ivproj(value)

h = size(ipq)[1] #h == hs * head
hs = div(h, mh.head)

#size(hq) == (hs, seq_len, head)
hq = permutedims(reshape(ipq, hs, mh.head, :), [1, 3, 2])
hk = permutedims(reshape(ipk, hs, mh.head, :), [1, 3, 2])
hv = permutedims(reshape(ipv, hs, mh.head, :), [1, 3, 2])

atten = attention1(hq, hk, hv,
mask,
mh.future,
mh.drop)

# size(atten) == (head*hs, seq_len)
atten = reshape(permutedims(atten, [1, 3, 2]), h, :)

mh.oproj(atten)
end

In [19]:
function apply_mask1(score, mask)
    s = size(score)
    ms = size(mask)
    bxn = s[end]
    b = ms[end]
    if bxn == b || b == 1
      return score .+ mask
    else
      return reshape(reshape(score, s[1:end-1]..., :, b) .+
                     reshape(mask, ms[1:end-1]..., 1, b), s)
    end
  end
  
  apply_mask1(score::AbstractArray{T}, ::Nothing, future) where T = future ? score : apply_mask1(score, create_atten_mask1(T, score, nothing, future))
  apply_mask1(score::AbstractArray{T}, mask, future) where T = apply_mask(score, create_atten_mask1(T, score, mask, future))

apply_mask1 (generic function with 3 methods)

In [20]:
# of type float (to allow for integer inputs)
function oftf(x, y)
    oftype(float(x), y)
end
function nelu(x, α=1)
    ifelse(x ≥ 0, float(x)+1, @fastmath oftf(x, α) * (exp(x) - 1)+1)
end

nelu (generic function with 2 methods)

In [21]:
x = embedding(encoded_sample)

512×12 Matrix{Float32}:
  0.549632   -0.522535    -0.998646   -0.661908   …  -0.00383869   0.848192
  0.768887    0.906328     0.219783   -0.686574      -0.95647     -0.845274
  0.569373   -0.418463    -0.956203   -0.74893       -0.370091     0.528791
  0.840675    0.950004     0.311458   -0.567978      -0.744703    -0.929366
  0.555678   -0.269035    -1.00508    -0.82683       -0.67915      0.222829
  0.72333     0.925027     0.486339   -0.434401   …  -0.434964    -0.961428
  0.617996   -0.180726    -0.93347    -1.04568       -1.04541     -0.129594
  0.744921    0.895699     0.471649   -0.304143      -0.0880868   -0.907648
  0.632271   -0.237903    -0.881746   -0.917644      -0.964045    -0.635149
  0.755272    1.0406       0.561783   -0.208443       0.223698    -0.592334
  ⋮                                               ⋱   ⋮           
 -0.0130005  -0.023082     0.0746827   0.0275085      0.0283168    0.0333594
  1.06702     0.973927     0.93401     1.0153         1.0153       1.015

In [22]:
function splitHeads(x, batch_size, head, depth)
    x = reshape(x, (batch_size, :, head, depth))
end

splitHeads (generic function with 1 method)

In [23]:
#hier anstatt softmax linear
function attention1(query::A1,
    key::A2,
    value::A3,
    mask, future::Bool,
    dropout) where {T,
                    A1 <: Abstract3DTensor{T},
                    A2 <: Abstract3DTensor{T},
                    A3 <: Abstract3DTensor{T}}
#size(query) == (dims, {q,k}_seq_len, batch) == size(key) == size(value)
#size(ipq) == (hs, q_seq_len, head * batch)
#size(score) == (k_seq_len, q_seq_len, batch)
#=
dk = size(key, 1)
score = batchedmul(key, query; transA = true) 
score = score ./ convert(T, sqrt(dk))

score = apply_mask1(score, mask, future)
score = softmax(score; dims=1)
dropout !== nothing && (score = dropout(score))
batchedmul(value, score) #size(return) == (dims, q_seq_len, batch)
=#
#size(query) == (batch, {q,k}_seq_len, dims) == size(key) == size(value)
query = permutedims(query,(3,2,1))
key = permutedims(key,(3,2,1))
value = permutedims(value,(3,2,1))

qs = size(query)
ks = size(key)
vs = size(value)

#depth = div(mh.size, mh.head)
depth = div(64, 8)   

#query = nelu(splitHeads(query, qs[1], mh.head, depth))
#key = nelu(splitHeads(key, qs[1], mh.head, depth))
#value = splitHeads(value, qs[1], mh.head, depth)

query = nelu.(splitHeads(query, qs[1], 8, depth))
key = nelu.(splitHeads(key, ks[1], 8, depth))
value = splitHeads(value, vs[1], 8, depth)
# size(k_v) == (batch_size, depth_k, depth_v, seq_len_v)
@tullio k_v[m,d,e,h] := key[m,j,h,d]*value[m,j,h,e]

k_reduced = dropdims(sum(key, dims=2), dims=2)

k_reduced = k_reduced .+ 1e-8

@tullio z_1[m,l,h] := query[m,l,h,d]*k_reduced[m,h,d]
# size(z) == (batch_size, num_heads, seq_len_q)
z = 1 ./ z_1 # ...
# size(output) == (batch_size,len_q, heads, depth_v)
@tullio output[m,l,h,e] := query[m,l,h,d]*k_v[m,d,e,h]*z[m,l,h]

#output = reshape(output, (qs[1], qs[2], mh.head*depth))
# size(output) == (batch_size,len_q, d_model)
output = reshape(output, (qs[1], :, 8*depth))

output = permutedims(output,(3,2,1))
#(dims, q_seq_len, batch)
end

attention1 (generic function with 1 method)

In [24]:
abstract type AbstractTransformer end

In [25]:
struct PwFFN{Di<:Dense, Do<:Dense}
    din::Di
    dout::Do
end

@functor PwFFN


"just a wrapper for two dense layer."
function PwFFN(size::Int, h::Int, act = nelu)
    PwFFN(
    Dense(size, h, act),
    Dense(h, size)
)
end
function (pw::PwFFN)(x::AbstractMatrix)
  #size(x) == (dims, seq_len)
  pw.dout(pw.din(x))
end

function (pw::PwFFN)(x::A) where {T, N, A<:AbstractArray{T, N}}
  new_x = reshape(x, size(x, 1), :)
  y = pw(new_x)
  return reshape(y, Base.setindex(size(x), size(y, 1), 1))
end

In [26]:
struct Transformer1{MA<:MultiheadAttention_1, LA<:LayerNorm, P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
    mh::MA
    mhn::LA
    pw::P
    pwn::LP
    drop::DP
end

@functor Transformer1


"""
    Transformer(size::Int, head::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)
    Transformer(size::Int, head::Int, hs::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)  

Transformer layer.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
When `future` is `false`, the k-th token can't see the j-th tokens where j > k. `pdrop` is the dropout rate.
"""

########
#ganze Funktion von Transformer ändern
########
function Transformer1(size::Int, head::Int, ps::Int; future::Bool = true, act = relu, pdrop = 0.1)  #relu vielleicht wieder auf elu ändern
    rem(size, head) != 0 && error("size not divisible by head")
    Transformer1(size, head, div(size, head), ps;future=future, act=act, pdrop=pdrop)
end

Transformer1(size::Int, head::Int, hs::Int, ps::Int; future::Bool = true, act = relu, pdrop = 0.1) = Transformer1(
    MultiheadAttention_1(head, size, hs, size; future=future, pdrop=pdrop),
    LayerNorm(size),
    PwFFN(size, ps, act),
    LayerNorm(size),
    Dropout(pdrop),     #braucht man das? vielleicht nicht
)

function (t::Transformer1)(x::A, mask=nothing) where {T, N, A<:AbstractArray{T, N}}
    dropout = t.drop
    a = t.mh(x, x, x; mask=mask)
    a = dropout(a)
    res_a = x + a
    res_a = t.mhn(res_a)
    pwffn = t.pw(res_a)
    pwffn = dropout(pwffn)
    res_pwffn = res_a + pwffn
    res_pwffn = t.pwn(res_pwffn)
    res_pwffn
end

function Base.show(io::IO, t::Transformer1) 
    hs = div(size(t.mh.iqproj.weight)[1], t.mh.head)
    h, ps = size(t.pw.dout.weight)

    print(io, "Transformer(")
    print(io, "head=$(t.mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "pwffn_size=$(ps), ")
    print(io, "size=$(h)")
    if Flux.istraining()
        print(io, ", dropout=$(t.drop.p))")
    else
        print(io, ")")
    end
end

In [27]:
###########
    #auch diese Funktion ändern
    ###############
    struct TransformerDecoder1{MA<:MultiheadAttention_1, LA<:LayerNorm,
        IMA<:MultiheadAttention_1, ILA<:LayerNorm,
        P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
mh::MA
mhn::LA
imh::IMA
imhn::ILA
pw::P
pwn::LP
drop::DP
end

@functor TransformerDecoder1

"""
TransformerDecoder(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
TransformerDecoder(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1)

TransformerDecoder layer. Decode the value from a Encoder.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
`pdrop` is the dropout rate.
"""
function TransformerDecoder1(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
rem(size, head) != 0 && error("size not divisible by head")
TransformerDecoder1(size, head, div(size, head), ps; act=act, pdrop=pdrop)
end

TransformerDecoder1(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1) = TransformerDecoder1(
MultiheadAttention_1(head, size, hs, size; future=false, pdrop=pdrop),
LayerNorm(size),
MultiheadAttention_1(head, size, hs, size; future=true, pdrop=pdrop), #future = true --> Unterschied zu oben??
LayerNorm(size),
PwFFN(size, ps, act),
LayerNorm(size),
Dropout(pdrop),
)

function (td::TransformerDecoder1)(x::AbstractArray{T,N}, m, mask=nothing) where {T,N}
dropout = td.drop
a = td.mh(x,x,x)
a = dropout(a)
res_a = x + a
res_a = td.mhn(res_a)

ia = td.imh(res_a, m, m, mask=mask)
ia = dropout(ia)
res_ia = res_a + ia
res_ia = td.imhn(res_ia)

pwffn = td.pw(res_ia)
pwffn = dropout(pwffn)
res_pwffn = res_ia + pwffn
res_pwffn = td.pwn(res_pwffn)
res_pwffn
end

function Base.show(io::IO, td::TransformerDecoder1)
hs = div(size(td.imh.iqproj.weight)[1], td.imh.head)
h, ps = size(td.pw.dout.weight)

print(io, "TransformerDecoder(")
print(io, "head=$(td.mh.head), ")
print(io, "head_size=$(hs), ")
print(io, "pwffn_size=$(ps), ")
print(io, "size=$(h)")
if Flux.istraining()
print(io, ", dropout=$(td.drop.p))")
else
print(io, ")")
end
end

In [28]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [29]:
encode_t2 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [30]:
encode_t3 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [31]:
encode_t4 = Transformer1(512, 8, 64, 2048) |> gpu


Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [32]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [33]:
decode_t2 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [34]:
decode_t3 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [35]:
decode_t4 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [36]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [37]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    t2 = encode_t2(t1)
    t3 = encode_t2(t2)
    t4 = encode_t2(t3)
    return t2
  end

encoder_forward (generic function with 1 method)

In [38]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    t2 = decode_t2(t1, m)
    t3 = decode_t3(t2, m)
    t4 = decode_t4(t3, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [39]:
#bleibt so
enc = encoder_forward(encoded_sample);

In [40]:
#bleibt so
probs = decoder_forward(encoded_sample, enc);

In [41]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [42]:
Flux.@nograd smooth

In [43]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    #l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])
    l = logcrossentropy(label[:, 2:end, :], probs[:, 1:end-1, :])  #VA: geändert
    return l
  end

loss (generic function with 1 method)

In [44]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, encode_t2, encode_t3, encode_t4, decode_t1, decode_t2, decode_t3, decode_t4, linear);

In [45]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [46]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:100
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [47]:
train!()


┌ Info: start training
└ @ Main /Users/johannes/Documents/GitHub/DM2022-LinearTransformers/test/LinAt.ipynb:5


loss = 82.16149829576104


loss = 79.2303654514007


loss = 77.76381240567338


loss = 77.61970393014899


...

loss = 77.00986577963972


loss = 77.47672778715487


loss = 76.92654794231954


loss = 76.84434542569203


loss = 76.96297547045943


In [48]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

translate (generic function with 1 method)

In [49]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

21-element Vector{String}:
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 ⋮
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"
 "7"